# Report
## Capstone Project - The Battle of Neighborhoods

## Table of content
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## 1. Introduction: Business Problem <a name="introduction"></a>


ABC is a Toronto-based event organization that is currently hosting a 7-day event for photographers from around the world. Businesses need to make a good plan, including hotels, meeting rooms, scenic spots, shops, restaurants and cafes. The company must list Toronto's attractions, including the closest restaurants, cafes, and shops at each location, and must provide maps in case the photographer is unable to use its mobile app for any reason.

## 2. Data <a name="data"></a>

The data used in this project is provided by Foursquare location data. The data is grouped by landscape area, and each area includes information about the area and all information about restaurants, cafes, and shops in the area.

In [2]:
### import libraries
import requests # to handle requests
import pandas as pd # for data analsysis
import numpy as np # to handle data in a vectorized manner

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
#tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.21.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [3]:
ClIENT_ID = 'REO5ATHOUEQPWAR5EM0C5EO5MIEBBIC2HPHONFC4M2QVGH4B' # your Foursquare ID
ClIENT_SECRET = 'CKMO5IVBABJ05DD4YGMLCI1YNHDDDHEV10JLQDN01CNAFTAG' # your Foursquare Secret
VERSION = '20180604'
LIMIT =30
print('Your credentails:')
print('Foursquare_ID: ' + ClIENT_ID)
print('Foursquare_Secret:' + ClIENT_SECRET)

Your credentails:
Foursquare_ID: REO5ATHOUEQPWAR5EM0C5EO5MIEBBIC2HPHONFC4M2QVGH4B
Foursquare_Secret:CKMO5IVBABJ05DD4YGMLCI1YNHDDDHEV10JLQDN01CNAFTAG


In [4]:
# define the city and get its latitude & longitude 
city = 'Toronto'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.653963 -79.387207


In [5]:
# search for hotels
search_query = 'Hotel'
radius = 500

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=REO5ATHOUEQPWAR5EM0C5EO5MIEBBIC2HPHONFC4M2QVGH4B&client_secret=CKMO5IVBABJ05DD4YGMLCI1YNHDDDHEV10JLQDN01CNAFTAG&ll=43.653963,-79.387207&v=20180604&query=Hotel&radius=500&limit=30'

In [6]:
# Send the GET Request and examine the results
results = requests.get(url).json()
#results

In [7]:
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4ab2d511f964a5209b6c20e3,123 Queen Street West,CA,Toronto,Canada,at York St.,432,"[123 Queen Street West (at York St.), Toronto ...","[{'label': 'display', 'lat': 43.6505944, 'lng'...",43.650594,-79.384530,NaN,M5H 2M9,ON,Sheraton Centre Toronto Hotel,v-1582457188,NaN
1,"[{'id': '4bf58dd8d48988d1e7931735', 'name': 'J...",False,4b68aed1f964a520de862be3,194 Queen St W,CA,Toronto,Canada,Queen & St. Patrick,400,"[194 Queen St W (Queen & St. Patrick), Toronto...","[{'label': 'display', 'lat': 43.65050475544005...",43.650505,-79.388577,NaN,M5V 1Z1,ON,The Rex Hotel Jazz & Blues Bar,v-1582457188,62225795
2,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4f343a31e4b0230a3b337a90,123 Test Drive,CA,Toronto,Canada,at somewhere St,500,"[123 Test Drive (at somewhere St), Toronto ON ...","[{'label': 'display', 'lat': 43.658434, 'lng':...",43.658434,-79.387894,NaN,M2M 2M2,ON,VFM Test Hotel,v-1582457188,NaN
3,"[{'id': '4bf58dd8d48988d121941735', 'name': 'L...",False,5d59d01867339e000897bc88,NaN,CA,Toronto,Canada,NaN,388,"[Toronto ON M5H 2M9, Canada]","[{'label': 'display', 'lat': 43.651063, 'lng':...",43.651063,-79.384527,NaN,M5H 2M9,ON,Sheraton Centre Hotel Club Lounge,v-1582457188,NaN
4,"[{'id': '56aa371be4b08b9a8d5734cf', 'name': 'B...",False,5545d07e498e2facac03f666,123 Queen Street West,CA,Toronto,Canada,at Bay Street,376,"[123 Queen Street West (at Bay Street), Toront...","[{'label': 'display', 'lat': 43.65119976208405...",43.651200,-79.384520,Financial District,NaN,ON,Sheraton Centre Toronto Hotel - Grand Ballroom,v-1582457188,NaN


#### Clean the dataframe

In [8]:
# keep only columns that include venue name, and anything that is associated with location
clean_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')]+ ['id']
clean_dataframe = dataframe.loc[:,clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
clean_dataframe['categories'] = clean_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_dataframe.columns = [column.split('.')[-1] for column in clean_dataframe.columns]

clean_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,CA,Toronto,Canada,at York St.,432,"[123 Queen Street West (at York St.), Toronto ...","[{'label': 'display', 'lat': 43.6505944, 'lng'...",43.650594,-79.384530,NaN,M5H 2M9,ON,4ab2d511f964a5209b6c20e3
1,The Rex Hotel Jazz & Blues Bar,Jazz Club,194 Queen St W,CA,Toronto,Canada,Queen & St. Patrick,400,"[194 Queen St W (Queen & St. Patrick), Toronto...","[{'label': 'display', 'lat': 43.65050475544005...",43.650505,-79.388577,NaN,M5V 1Z1,ON,4b68aed1f964a520de862be3
2,VFM Test Hotel,Hotel,123 Test Drive,CA,Toronto,Canada,at somewhere St,500,"[123 Test Drive (at somewhere St), Toronto ON ...","[{'label': 'display', 'lat': 43.658434, 'lng':...",43.658434,-79.387894,NaN,M2M 2M2,ON,4f343a31e4b0230a3b337a90
3,Sheraton Centre Hotel Club Lounge,Lounge,NaN,CA,Toronto,Canada,NaN,388,"[Toronto ON M5H 2M9, Canada]","[{'label': 'display', 'lat': 43.651063, 'lng':...",43.651063,-79.384527,NaN,M5H 2M9,ON,5d59d01867339e000897bc88
4,Sheraton Centre Toronto Hotel - Grand Ballroom,Ballroom,123 Queen Street West,CA,Toronto,Canada,at Bay Street,376,"[123 Queen Street West (at Bay Street), Toront...","[{'label': 'display', 'lat': 43.65119976208405...",43.651200,-79.384520,Financial District,NaN,ON,5545d07e498e2facac03f666


In [9]:
# delete unnecessary columns
clean_dataframe2= clean_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs','neighborhood', 'id'], axis=1)
clean_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
1,The Rex Hotel Jazz & Blues Bar,Jazz Club,194 Queen St W,43.650505,-79.388577,M5V 1Z1,ON
2,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON
3,Sheraton Centre Hotel Club Lounge,Lounge,NaN,43.651063,-79.384527,M5H 2M9,ON
4,Sheraton Centre Toronto Hotel - Grand Ballroom,Ballroom,123 Queen Street West,43.651200,-79.384520,NaN,ON
5,Stathcona Hotel,Hotel,NaN,43.654947,-79.386359,NaN,ON
6,DoubleTree by Hilton,Hotel,108 Chestnut Street,43.654608,-79.385942,M5G 1R3,ON
7,650 Hotel,Hotel,650 Bay Street,43.657046,-79.384411,NaN,ON
8,Be SixFifty Hotel,Hotel,650 Bay Street,43.657120,-79.384560,NaN,ON
9,Hilton,Hotel,145 Richmond St W,43.649946,-79.385479,M5H 2L2,ON


In [10]:
# delete rows with none values
clean_dataframe3 = clean_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_dataframe3

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
1,The Rex Hotel Jazz & Blues Bar,Jazz Club,194 Queen St W,43.650505,-79.388577,M5V 1Z1,ON
2,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON
6,DoubleTree by Hilton,Hotel,108 Chestnut Street,43.654608,-79.385942,M5G 1R3,ON
9,Hilton,Hotel,145 Richmond St W,43.649946,-79.385479,M5H 2L2,ON
10,Shangri-La Toronto,Hotel,188 University Ave.,43.649129,-79.386557,M5H 0A3,ON
11,Marriott Downtown at CF Toronto Eaton Centre,Hotel,525 Bay Street,43.654728,-79.382422,M5G 2L2,ON
13,89 Chestnut Residence,College Residence Hall,89 Chestnut St.,43.654160,-79.385291,M5G 1R1,ON
14,Fitness Centre,Gym,Sheraton Centre,43.650985,-79.384002,M5H 2M9,ON
15,Fitness Centre,Gym,525 Bay St.,43.654690,-79.381739,M5G 2L2,ON


In [11]:
# delete rows which its category is not Hotel or Event Space
array= ['Hotel', 'Event Space']
hotel_dataframe= clean_dataframe3.loc[clean_dataframe3['categories'].isin(array)]
hotel_dataframe

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
2,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON
6,DoubleTree by Hilton,Hotel,108 Chestnut Street,43.654608,-79.385942,M5G 1R3,ON
9,Hilton,Hotel,145 Richmond St W,43.649946,-79.385479,M5H 2L2,ON
10,Shangri-La Toronto,Hotel,188 University Ave.,43.649129,-79.386557,M5H 0A3,ON
11,Marriott Downtown at CF Toronto Eaton Centre,Hotel,525 Bay Street,43.654728,-79.382422,M5G 2L2,ON
16,Grand Ballroom,Event Space,123 Queen St. W,43.651217,-79.383771,M5H 2M9,ON


In [12]:
# delete rows which has duplicate hotel's name
df_hotels = hotel_dataframe.drop_duplicates(subset='name', keep="first")
df_hotels

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
2,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON
6,DoubleTree by Hilton,Hotel,108 Chestnut Street,43.654608,-79.385942,M5G 1R3,ON
9,Hilton,Hotel,145 Richmond St W,43.649946,-79.385479,M5H 2L2,ON
10,Shangri-La Toronto,Hotel,188 University Ave.,43.649129,-79.386557,M5H 0A3,ON
11,Marriott Downtown at CF Toronto Eaton Centre,Hotel,525 Bay Street,43.654728,-79.382422,M5G 2L2,ON
16,Grand Ballroom,Event Space,123 Queen St. W,43.651217,-79.383771,M5H 2M9,ON


In [13]:
# choose the hotel which has the same postalCode with the event space
df_hotel = df_hotels[df_hotels.postalCode == 'M5H 2M9']
df_hotel

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
16,Grand Ballroom,Event Space,123 Queen St. W,43.651217,-79.383771,M5H 2M9,ON


#### search for park

In [14]:
# search for Parks
search_query = 'Park'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=REO5ATHOUEQPWAR5EM0C5EO5MIEBBIC2HPHONFC4M2QVGH4B&client_secret=CKMO5IVBABJ05DD4YGMLCI1YNHDDDHEV10JLQDN01CNAFTAG&ll=43.653963,-79.387207&v=20180604&query=Park&radius=10000&limit=30'

In [15]:
# Send the GET Request and examine the results
presults = requests.get(url).json()
#presults

In [16]:
# assign relevant part of JSON to venues
venues = presults['response']['venues']

# tranform venues into a dataframe
park_dataframe = json_normalize(venues)
park_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1fd931735', 'name': 'M...",False,4b170e47f964a5208cc123e3,691 University Ave,CA,Toronto,Canada,at College St,716,"[691 University Ave (at College St), Toronto O...","[{'label': 'display', 'lat': 43.66000635700919...",43.660006,-79.390272,M5G 2P1,ON,Queen's Park Subway Station,v-1582457291,NaN
1,"[{'id': '4d954b06a243a5684965b473', 'name': 'R...",False,4d3a1d23325ff04db2dd1845,763 Bay St.,CA,Toronto,Canada,btwn. College St. and Gerrard St.,672,[763 Bay St. (btwn. College St. and Gerrard St...,"[{'label': 'display', 'lat': 43.65982157793473...",43.659822,-79.385159,M5G 2R3,ON,Residences at College Park North,v-1582457291,NaN
2,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,4bfc5019da7120a132df49fd,450 Adelaide Street West,CA,Toronto,Canada,Brant St & Adelaide St W,1183,[450 Adelaide Street West (Brant St & Adelaide...,"[{'label': 'display', 'lat': 43.64738759338116...",43.647388,-79.398752,NaN,ON,St. Andrews Playground / Dog Park,v-1582457291,NaN
3,"[{'id': '4d954b06a243a5684965b473', 'name': 'R...",False,4ad80fcef964a520a61021e3,280 Simcoe St,CA,Toronto,Canada,Dundas Street West,282,"[280 Simcoe St (Dundas Street West), Toronto O...","[{'label': 'display', 'lat': 43.65544241747622...",43.655442,-79.390056,M5T 2Y5,ON,One Park Lane,v-1582457291,NaN
4,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,4b9d206bf964a520e69136e3,University Ave.,CA,Toronto,Canada,at Wellesley Ave.,1181,"[University Ave. (at Wellesley Ave.), Toronto ...","[{'label': 'display', 'lat': 43.66394609897775...",43.663946,-79.392180,M5R 2E8,ON,Queen's Park,v-1582457291,NaN


#### clean park

In [17]:
# keep only columns that include venue name, and anything that is associated with location
park_clean_columns = ['name', 'categories'] + [col for col in park_dataframe.columns if col.startswith('location.')]+ ['id']
clean_park_dataframe = park_dataframe.loc[:,park_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list1 = row['categories']
    except:
        categories_list1 = row['venue.categories']
        
    if len(categories_list1) == 0:
        return None
    else:
        return categories_list1[0]['name']

# filter the category for each row
clean_park_dataframe['categories'] = clean_park_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_park_dataframe.columns = [column.split('.')[-1] for column in clean_park_dataframe.columns]

clean_park_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Queen's Park Subway Station,Metro Station,691 University Ave,CA,Toronto,Canada,at College St,716,"[691 University Ave (at College St), Toronto O...","[{'label': 'display', 'lat': 43.66000635700919...",43.660006,-79.390272,M5G 2P1,ON,4b170e47f964a5208cc123e3
1,Residences at College Park North,Residential Building (Apartment / Condo),763 Bay St.,CA,Toronto,Canada,btwn. College St. and Gerrard St.,672,[763 Bay St. (btwn. College St. and Gerrard St...,"[{'label': 'display', 'lat': 43.65982157793473...",43.659822,-79.385159,M5G 2R3,ON,4d3a1d23325ff04db2dd1845
2,St. Andrews Playground / Dog Park,Park,450 Adelaide Street West,CA,Toronto,Canada,Brant St & Adelaide St W,1183,[450 Adelaide Street West (Brant St & Adelaide...,"[{'label': 'display', 'lat': 43.64738759338116...",43.647388,-79.398752,NaN,ON,4bfc5019da7120a132df49fd
3,One Park Lane,Residential Building (Apartment / Condo),280 Simcoe St,CA,Toronto,Canada,Dundas Street West,282,"[280 Simcoe St (Dundas Street West), Toronto O...","[{'label': 'display', 'lat': 43.65544241747622...",43.655442,-79.390056,M5T 2Y5,ON,4ad80fcef964a520a61021e3
4,Queen's Park,Park,University Ave.,CA,Toronto,Canada,at Wellesley Ave.,1181,"[University Ave. (at Wellesley Ave.), Toronto ...","[{'label': 'display', 'lat': 43.66394609897775...",43.663946,-79.392180,M5R 2E8,ON,4b9d206bf964a520e69136e3


In [18]:
# delete unnecessary columns
clean_park_dataframe2= clean_park_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_park_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Queen's Park Subway Station,Metro Station,691 University Ave,43.660006,-79.390272,M5G 2P1,ON
1,Residences at College Park North,Residential Building (Apartment / Condo),763 Bay St.,43.659822,-79.385159,M5G 2R3,ON
2,St. Andrews Playground / Dog Park,Park,450 Adelaide Street West,43.647388,-79.398752,NaN,ON
3,One Park Lane,Residential Building (Apartment / Condo),280 Simcoe St,43.655442,-79.390056,M5T 2Y5,ON
4,Queen's Park,Park,University Ave.,43.663946,-79.392180,M5R 2E8,ON
5,Berczy Park,Park,35 Wellington St. East,43.648048,-79.375172,NaN,ON
6,Grange Park,Park,NaN,43.652488,-79.392053,NaN,ON
7,TTC Streetcar #403 - Victoria park,Light Rail Station,NaN,43.650735,-79.386921,NaN,ON
8,Crunch Fitness - College Park,Gym / Fitness Center,382 Yonge Street,43.659263,-79.382632,M5G 1S8,ON
9,Clarence Square Park,Park,Spadina Ave,43.644222,-79.394230,NaN,ON


In [19]:
# delete rows with none values
clean_park_dataframe3 = clean_park_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_park_dataframe3

,name,categories,address,lat,lng,postalCode,state
0,Queen's Park Subway Station,Metro Station,691 University Ave,43.660006,-79.390272,M5G 2P1,ON
1,Residences at College Park North,Residential Building (Apartment / Condo),763 Bay St.,43.659822,-79.385159,M5G 2R3,ON
3,One Park Lane,Residential Building (Apartment / Condo),280 Simcoe St,43.655442,-79.390056,M5T 2Y5,ON
4,Queen's Park,Park,University Ave.,43.663946,-79.392180,M5R 2E8,ON
8,Crunch Fitness - College Park,Gym / Fitness Center,382 Yonge Street,43.659263,-79.382632,M5G 1S8,ON
10,Physiomed College Park,Medical Center,382 Yonge St,43.659240,-79.382860,M5B 1S8,ON
13,Canoe Landing Park,Park,50 Fort York Blvd,43.638762,-79.397067,M5V 3Z1,ON
14,Bellevue Square Park,Park,btwn Bellevue & Augusta Ave.,43.653610,-79.402199,M5T 2N4,ON
15,College Park,Shopping Mall,444 Yonge St,43.661237,-79.383603,M5B 2H4,ON
19,Victoria Park Subway Station,Metro Station,777 Victoria Park Ave,43.694883,-79.288735,M4B 2E2,ON


In [46]:
# delete rows which its category is not Park
df_park = clean_park_dataframe3[clean_park_dataframe3.categories == 'Park']
df_park

,name,categories,address,lat,lng,postalCode,state
4,Queen's Park,Park,University Ave.,43.663946,-79.392180,M5R 2E8,ON
13,Canoe Landing Park,Park,50 Fort York Blvd,43.638762,-79.397067,M5V 3Z1,ON
14,Bellevue Square Park,Park,btwn Bellevue & Augusta Ave.,43.653610,-79.402199,M5T 2N4,ON
21,High Park,Park,1873 Bloor St. W,43.646479,-79.463425,M6R 2Z3,ON
23,Trinity Bellwoods Park,Park,1053 Dundas St. W.,43.647072,-79.413756,M5H 2N2,ON
25,Barbara Hall Park,Park,519 Church St,43.666879,-79.381068,M4Y 2K9,ON


#### Search for Restaurants

In [20]:
# search for Restaurants
search_query = 'Restaurant'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=REO5ATHOUEQPWAR5EM0C5EO5MIEBBIC2HPHONFC4M2QVGH4B&client_secret=CKMO5IVBABJ05DD4YGMLCI1YNHDDDHEV10JLQDN01CNAFTAG&ll=43.653963,-79.387207&v=20180604&query=Restaurant&radius=10000&limit=30'

In [21]:
# Send the GET Request and examine the results
Rresults = requests.get(url).json()
#Rresults

In [22]:
# assign relevant part of JSON to venues
venues = Rresults['response']['venues']

# tranform venues into a dataframe
Restaurant_dataframe = json_normalize(venues)
Restaurant_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",False,4ad4c05ff964a52048f720e3,110 Chestnut Street,CA,Toronto,Canada,NaN,145,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]","[{'label': 'display', 'lat': 43.65488413420439...",43.654884,-79.385931,NaN,M5G 1R3,ON,Hemispheres Restaurant & Bistro,v-1582457518,NaN
1,"[{'id': '4bf58dd8d48988d1f5931735', 'name': 'D...",False,4ad4c060f964a5207ff720e3,323 Spadina Ave.,CA,Toronto,Canada,at D'Arcy St.,922,"[323 Spadina Ave. (at D'Arcy St.), Toronto ON ...","[{'label': 'display', 'lat': 43.65431754076345...",43.654318,-79.398650,Kensington Market,M5T 2E9,ON,Rol San Restaurant 龍笙棧,v-1582457518,NaN
2,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",False,4ae29812f964a520288f21e3,309 Spadina Ave.,CA,Toronto,Canada,btwn Dundas St. W & D'Arcy St.,896,[309 Spadina Ave. (btwn Dundas St. W & D'Arcy ...,"[{'label': 'display', 'lat': 43.65386562507761...",43.653866,-79.398334,NaN,M5T 2E6,ON,Swatow Restaurant 汕頭小食家,v-1582457518,NaN
3,"[{'id': '4bf58dd8d48988d1d1941735', 'name': 'N...",False,4b266f05f964a520657b24e3,266 Spadina Ave,CA,Toronto,Canada,at Willison Sq,892,"[266 Spadina Ave (at Willison Sq), Toronto ON ...","[{'label': 'display', 'lat': 43.6522783893466,...",43.652278,-79.398039,NaN,M5T 2E4,ON,Goldstone Noodle Restaurant 金石,v-1582457518,NaN
4,"[{'id': '4bf58dd8d48988d123941735', 'name': 'W...",False,4ad4c05cf964a520dff520e3,301 Front St W,CA,Toronto,Canada,301 Front St. W,1271,"[301 Front St W (301 Front St. W), Toronto ON ...","[{'label': 'display', 'lat': 43.64253731714456...",43.642537,-79.387042,NaN,M5V 2T6,ON,360 Restaurant,v-1582457518,NaN


#### clean restaurant

In [23]:
# keep only columns that include venue name, and anything that is associated with location
Restaurant_clean_columns = ['name', 'categories'] + [col for col in Restaurant_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Restaurant_dataframe = Restaurant_dataframe.loc[:,Restaurant_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list3 = row['categories']
    except:
        categories_list3 = row['venue.categories']
        
    if len(categories_list3) == 0:
        return None
    else:
        return categories_list3[0]['name']

# filter the category for each row
clean_Restaurant_dataframe['categories'] = clean_Restaurant_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Restaurant_dataframe.columns = [column.split('.')[-1] for column in clean_Restaurant_dataframe.columns]

clean_Restaurant_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,CA,Toronto,Canada,NaN,145,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]","[{'label': 'display', 'lat': 43.65488413420439...",43.654884,-79.385931,NaN,M5G 1R3,ON,4ad4c05ff964a52048f720e3
1,Rol San Restaurant 龍笙棧,Dim Sum Restaurant,323 Spadina Ave.,CA,Toronto,Canada,at D'Arcy St.,922,"[323 Spadina Ave. (at D'Arcy St.), Toronto ON ...","[{'label': 'display', 'lat': 43.65431754076345...",43.654318,-79.398650,Kensington Market,M5T 2E9,ON,4ad4c060f964a5207ff720e3
2,Swatow Restaurant 汕頭小食家,Chinese Restaurant,309 Spadina Ave.,CA,Toronto,Canada,btwn Dundas St. W & D'Arcy St.,896,[309 Spadina Ave. (btwn Dundas St. W & D'Arcy ...,"[{'label': 'display', 'lat': 43.65386562507761...",43.653866,-79.398334,NaN,M5T 2E6,ON,4ae29812f964a520288f21e3
3,Goldstone Noodle Restaurant 金石,Noodle House,266 Spadina Ave,CA,Toronto,Canada,at Willison Sq,892,"[266 Spadina Ave (at Willison Sq), Toronto ON ...","[{'label': 'display', 'lat': 43.6522783893466,...",43.652278,-79.398039,NaN,M5T 2E4,ON,4b266f05f964a520657b24e3
4,360 Restaurant,Wine Bar,301 Front St W,CA,Toronto,Canada,301 Front St. W,1271,"[301 Front St W (301 Front St. W), Toronto ON ...","[{'label': 'display', 'lat': 43.64253731714456...",43.642537,-79.387042,NaN,M5V 2T6,ON,4ad4c05cf964a520dff520e3


In [24]:
# delete unnecessary columns
clean_Restaurant_dataframe2= clean_Restaurant_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'neighborhood', 'id'], axis=1)
clean_Restaurant_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,M5G 1R3,ON
1,Rol San Restaurant 龍笙棧,Dim Sum Restaurant,323 Spadina Ave.,43.654318,-79.398650,M5T 2E9,ON
2,Swatow Restaurant 汕頭小食家,Chinese Restaurant,309 Spadina Ave.,43.653866,-79.398334,M5T 2E6,ON
3,Goldstone Noodle Restaurant 金石,Noodle House,266 Spadina Ave,43.652278,-79.398039,M5T 2E4,ON
4,360 Restaurant,Wine Bar,301 Front St W,43.642537,-79.387042,M5V 2T6,ON
5,New Sky Restaurant 小沙田食家,Chinese Restaurant,353 Spadina Ave.,43.655337,-79.398897,M5T 2G3,ON
6,Victoria's Restaurant,Restaurant,37 King Street East,43.649298,-79.376431,M5C 1E9,ON
7,Azure Restaurant & Bar,Restaurant,225 Front St W,43.644749,-79.385113,M5V 2X3,ON
8,Some Time BBQ Grill Restaurant 碳烤屋,Szechuan Restaurant,988 Baldwin Street,43.655874,-79.393826,NaN,ON
9,Victor Restaurant & Bar,Bar,30 Mercer Street,43.645634,-79.391125,M5V 1H3,ON


In [25]:
# delete rows with none values
df_Restaurant = clean_Restaurant_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Restaurant

,name,categories,address,lat,lng,postalCode,state
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,M5G 1R3,ON
1,Rol San Restaurant 龍笙棧,Dim Sum Restaurant,323 Spadina Ave.,43.654318,-79.398650,M5T 2E9,ON
2,Swatow Restaurant 汕頭小食家,Chinese Restaurant,309 Spadina Ave.,43.653866,-79.398334,M5T 2E6,ON
3,Goldstone Noodle Restaurant 金石,Noodle House,266 Spadina Ave,43.652278,-79.398039,M5T 2E4,ON
4,360 Restaurant,Wine Bar,301 Front St W,43.642537,-79.387042,M5V 2T6,ON
5,New Sky Restaurant 小沙田食家,Chinese Restaurant,353 Spadina Ave.,43.655337,-79.398897,M5T 2G3,ON
6,Victoria's Restaurant,Restaurant,37 King Street East,43.649298,-79.376431,M5C 1E9,ON
7,Azure Restaurant & Bar,Restaurant,225 Front St W,43.644749,-79.385113,M5V 2X3,ON
9,Victor Restaurant & Bar,Bar,30 Mercer Street,43.645634,-79.391125,M5V 1H3,ON
10,Tasty's Caribbean Restaurant & Catering,Caribbean Restaurant,405 Spadina Ave,43.656794,-79.399251,M5T 2G6,ON


#### search for cafe

In [26]:
# search for Cafeteria
search_query = 'Cafeteria'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=REO5ATHOUEQPWAR5EM0C5EO5MIEBBIC2HPHONFC4M2QVGH4B&client_secret=CKMO5IVBABJ05DD4YGMLCI1YNHDDDHEV10JLQDN01CNAFTAG&ll=43.653963,-79.387207&v=20180604&query=Cafeteria&radius=10000&limit=30'

In [27]:
# Send the GET Request and examine the results
cresults = requests.get(url).json()
#cresults

In [28]:
# assign relevant part of JSON to venues
venues = cresults['response']['venues']

# tranform venues into a dataframe
Cafeteria_dataframe = json_normalize(venues)
Cafeteria_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",False,4f9165bde4b04fef0087e4e2,The Hospital for Sick Children (SickKids),CA,Toronto,Canada,NaN,372,"[The Hospital for Sick Children (SickKids), To...","[{'label': 'display', 'lat': 43.65720930549997...",43.657209,-79.386063,NaN,ON,Cafeteria,v-1582457608
1,"[{'id': '4bf58dd8d48988d142941735', 'name': 'A...",False,528e99a211d262edc3708a6f,388 Spadina Ave,CA,Toronto,Canada,at Nassau St,977,"[388 Spadina Ave (at Nassau St), Toronto ON M5...","[{'label': 'display', 'lat': 43.65545427446646...",43.655454,-79.399163,M5T 2G5,ON,Stay Cafeteria 慢走,v-1582457608
2,"[{'id': '4bf58dd8d48988d128941735', 'name': 'C...",False,53bd6dfe498e6c1d3bc9333e,NaN,CA,Toronto,Canada,NaN,260,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.655657, 'lng':...",43.655657,-79.389443,NaN,ON,Industrial Alliance Cafeteria,v-1582457608
3,"[{'id': '4bf58dd8d48988d1d0941735', 'name': 'D...",False,556f826b498e07f469e6bf1f,NaN,CA,NaN,Canada,NaN,431,[Canada],"[{'label': 'display', 'lat': 43.65417861938476...",43.654179,-79.392555,NaN,NaN,Dears Cafeteria,v-1582457608
4,"[{'id': '4bf58dd8d48988d1a1941735', 'name': 'C...",False,4ec2b68a8231a83de8c45191,100 McCaul St.,CA,Toronto,Canada,Second Floor,389,"[100 McCaul St. (Second Floor), Toronto ON, Ca...","[{'label': 'display', 'lat': 43.65278016892648...",43.652780,-79.391762,NaN,ON,OCAD Cafeteria,v-1582457608


#### Clean Cafeteria

In [29]:
# keep only columns that include venue name, and anything that is associated with location
Cafeteria_clean_columns = ['name', 'categories'] + [col for col in Cafeteria_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Cafeteria_dataframe = Cafeteria_dataframe.loc[:,Cafeteria_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list4 = row['categories']
    except:
        categories_list4 = row['venue.categories']
        
    if len(categories_list4) == 0:
        return None
    else:
        return categories_list4[0]['name']

# filter the category for each row
clean_Cafeteria_dataframe['categories'] = clean_Cafeteria_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Cafeteria_dataframe.columns = [column.split('.')[-1] for column in clean_Cafeteria_dataframe.columns]

clean_Cafeteria_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Cafeteria,Fast Food Restaurant,The Hospital for Sick Children (SickKids),CA,Toronto,Canada,NaN,372,"[The Hospital for Sick Children (SickKids), To...","[{'label': 'display', 'lat': 43.65720930549997...",43.657209,-79.386063,NaN,ON,4f9165bde4b04fef0087e4e2
1,Stay Cafeteria 慢走,Asian Restaurant,388 Spadina Ave,CA,Toronto,Canada,at Nassau St,977,"[388 Spadina Ave (at Nassau St), Toronto ON M5...","[{'label': 'display', 'lat': 43.65545427446646...",43.655454,-79.399163,M5T 2G5,ON,528e99a211d262edc3708a6f
2,Industrial Alliance Cafeteria,Cafeteria,NaN,CA,Toronto,Canada,NaN,260,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.655657, 'lng':...",43.655657,-79.389443,NaN,ON,53bd6dfe498e6c1d3bc9333e
3,Dears Cafeteria,Dessert Shop,NaN,CA,NaN,Canada,NaN,431,[Canada],"[{'label': 'display', 'lat': 43.65417861938476...",43.654179,-79.392555,NaN,NaN,556f826b498e07f469e6bf1f
4,OCAD Cafeteria,College Cafeteria,100 McCaul St.,CA,Toronto,Canada,Second Floor,389,"[100 McCaul St. (Second Floor), Toronto ON, Ca...","[{'label': 'display', 'lat': 43.65278016892648...",43.652780,-79.391762,NaN,ON,4ec2b68a8231a83de8c45191


In [30]:
# delete unnecessary columns
clean_Cafeteria_dataframe2= clean_Cafeteria_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_Cafeteria_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Cafeteria,Fast Food Restaurant,The Hospital for Sick Children (SickKids),43.657209,-79.386063,NaN,ON
1,Stay Cafeteria 慢走,Asian Restaurant,388 Spadina Ave,43.655454,-79.399163,M5T 2G5,ON
2,Industrial Alliance Cafeteria,Cafeteria,NaN,43.655657,-79.389443,NaN,ON
3,Dears Cafeteria,Dessert Shop,NaN,43.654179,-79.392555,NaN,NaN
4,OCAD Cafeteria,College Cafeteria,100 McCaul St.,43.652780,-79.391762,NaN,ON
5,Chestnut Tree Cafeteria,Cafeteria,89 Chestnut Street,43.648231,-79.384045,NaN,ON
6,Sears Cafeteria,Food Court,Yonge,43.656038,-79.380672,NaN,ON
7,De La Salle College Cafeteria,College Cafeteria,131 Farnham Avenue,43.683003,-79.397815,M4V 1H7,ON
8,Med Sci Cafeteria,College Cafeteria,NaN,43.660260,-79.392977,NaN,ON
9,Ryerson Hub Cafeteria,College Cafeteria,350 Victoria St.,43.658475,-79.377653,NaN,ON


In [31]:
# delete unnecessary columns
clean_Cafeteria_dataframe2= clean_Cafeteria_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_Cafeteria_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Cafeteria,Fast Food Restaurant,The Hospital for Sick Children (SickKids),43.657209,-79.386063,NaN,ON
1,Stay Cafeteria 慢走,Asian Restaurant,388 Spadina Ave,43.655454,-79.399163,M5T 2G5,ON
2,Industrial Alliance Cafeteria,Cafeteria,NaN,43.655657,-79.389443,NaN,ON
3,Dears Cafeteria,Dessert Shop,NaN,43.654179,-79.392555,NaN,NaN
4,OCAD Cafeteria,College Cafeteria,100 McCaul St.,43.652780,-79.391762,NaN,ON
5,Chestnut Tree Cafeteria,Cafeteria,89 Chestnut Street,43.648231,-79.384045,NaN,ON
6,Sears Cafeteria,Food Court,Yonge,43.656038,-79.380672,NaN,ON
7,De La Salle College Cafeteria,College Cafeteria,131 Farnham Avenue,43.683003,-79.397815,M4V 1H7,ON
8,Med Sci Cafeteria,College Cafeteria,NaN,43.660260,-79.392977,NaN,ON
9,Ryerson Hub Cafeteria,College Cafeteria,350 Victoria St.,43.658475,-79.377653,NaN,ON


In [41]:
# delete rows with none values
df_Cafeteria = clean_Cafeteria_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Cafeteria

,name,categories,address,lat,lng,postalCode,state
1,Stay Cafeteria 慢走,Asian Restaurant,388 Spadina Ave,43.655454,-79.399163,M5T 2G5,ON
7,De La Salle College Cafeteria,College Cafeteria,131 Farnham Avenue,43.683003,-79.397815,M4V 1H7,ON
19,The Hub,College Cafeteria,350 Victoria St.,43.658585,-79.380622,M5B 2K3,ON
21,Marketeria,Restaurant,30 Bond St.,43.653585,-79.378843,M5B 1W8,ON
27,6th Floor Cafeteria 12 Concorde Place,Café,12 Concorde Place,43.721861,-79.329339,m3c 3k7,ON


#### search shopping center

In [32]:
# search for Shopping
search_query = 'Shopping'
radius = 1000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=REO5ATHOUEQPWAR5EM0C5EO5MIEBBIC2HPHONFC4M2QVGH4B&client_secret=CKMO5IVBABJ05DD4YGMLCI1YNHDDDHEV10JLQDN01CNAFTAG&ll=43.653963,-79.387207&v=20180604&query=Shopping&radius=1000&limit=30'

In [33]:
# Send the GET Request and examine the results
sresults = requests.get(url).json()
#sresults

In [35]:
# assign relevant part of JSON to venues
venues = sresults['response']['venues']

# tranform venues into a dataframe
Shopping_dataframe = json_normalize(venues)
Shopping_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,4ad4c062f964a520fcf720e3,280 Spadina Ave,CA,Toronto,Canada,at Dundas St W,897,"[280 Spadina Ave (at Dundas St W), Toronto ON ...","[{'label': 'display', 'lat': 43.65277367198862...",43.652774,-79.398222,NaN,M5T 3A5,ON,Dragon City Shopping Mall 龍城,v-1582457899
1,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,4ad77a12f964a520260b21e3,220 Yonge St,CA,Toronto,Canada,btwn Queen & Dundas,538,"[220 Yonge St (btwn Queen & Dundas), Toronto O...",NaN,43.653534,-79.380551,"Downtown Toronto, Toronto, ON",M5B 2H1,ON,CF Toronto Eaton Centre,v-1582457899
2,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,4b9e86dff964a520e6eb36e3,66 Wellington St W,CA,Toronto,Canada,btw York & Bay St,908,"[66 Wellington St W (btw York & Bay St), Toron...","[{'label': 'display', 'lat': 43.64718424611984...",43.647184,-79.380932,NaN,M5K 1A1,ON,TD Centre Shopping Concourse,v-1582457899
3,"[{'id': '4bf58dd8d48988d1f6941735', 'name': 'D...",False,5840cafe06f1a34af80cc609,176 Yonge Street,CA,Toronto,Canada,Queen Street West,688,"[176 Yonge Street (Queen Street West), Toronto...","[{'label': 'display', 'lat': 43.65181, 'lng': ...",43.651810,-79.379192,NaN,M5C 2L7,ON,Saks Fifth Avenue Club - Personal Shopping,v-1582457899
4,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",False,4b83d653f964a520d91231e3,461 King St. West,CA,Toronto,Canada,NaN,974,"[461 King St. West, Toronto ON M5V 1K4, Canada]","[{'label': 'display', 'lat': 43.64795765580337...",43.647958,-79.396006,NaN,M5V 1K4,ON,Yellow Pages - RedFlagDeals.com,v-1582457899


#### clean shopping center

In [37]:
# keep only columns that include venue name, and anything that is associated with location
Shopping_clean_columns = ['name', 'categories'] + [col for col in Shopping_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Shopping_dataframe = Shopping_dataframe.loc[:,Shopping_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list5 = row['categories']
    except:
        categories_list5 = row['venue.categories']
        
    if len(categories_list5) == 0:
        return None
    else:
        return categories_list5[0]['name']

# filter the category for each row
clean_Shopping_dataframe['categories'] = clean_Shopping_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Shopping_dataframe.columns = [column.split('.')[-1] for column in clean_Shopping_dataframe.columns]

clean_Shopping_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Dragon City Shopping Mall 龍城,Shopping Mall,280 Spadina Ave,CA,Toronto,Canada,at Dundas St W,897,"[280 Spadina Ave (at Dundas St W), Toronto ON ...","[{'label': 'display', 'lat': 43.65277367198862...",43.652774,-79.398222,NaN,M5T 3A5,ON,4ad4c062f964a520fcf720e3
1,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,CA,Toronto,Canada,btwn Queen & Dundas,538,"[220 Yonge St (btwn Queen & Dundas), Toronto O...",NaN,43.653534,-79.380551,"Downtown Toronto, Toronto, ON",M5B 2H1,ON,4ad77a12f964a520260b21e3
2,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,CA,Toronto,Canada,btw York & Bay St,908,"[66 Wellington St W (btw York & Bay St), Toron...","[{'label': 'display', 'lat': 43.64718424611984...",43.647184,-79.380932,NaN,M5K 1A1,ON,4b9e86dff964a520e6eb36e3
3,Saks Fifth Avenue Club - Personal Shopping,Department Store,176 Yonge Street,CA,Toronto,Canada,Queen Street West,688,"[176 Yonge Street (Queen Street West), Toronto...","[{'label': 'display', 'lat': 43.65181, 'lng': ...",43.651810,-79.379192,NaN,M5C 2L7,ON,5840cafe06f1a34af80cc609
4,Yellow Pages - RedFlagDeals.com,Office,461 King St. West,CA,Toronto,Canada,NaN,974,"[461 King St. West, Toronto ON M5V 1K4, Canada]","[{'label': 'display', 'lat': 43.64795765580337...",43.647958,-79.396006,NaN,M5V 1K4,ON,4b83d653f964a520d91231e3


In [38]:
# delete unnecessary columns
clean_Shopping_dataframe2= clean_Shopping_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'neighborhood', 'id'], axis=1)
clean_Shopping_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Dragon City Shopping Mall 龍城,Shopping Mall,280 Spadina Ave,43.652774,-79.398222,M5T 3A5,ON
1,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,43.653534,-79.380551,M5B 2H1,ON
2,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,43.647184,-79.380932,M5K 1A1,ON
3,Saks Fifth Avenue Club - Personal Shopping,Department Store,176 Yonge Street,43.651810,-79.379192,M5C 2L7,ON
4,Yellow Pages - RedFlagDeals.com,Office,461 King St. West,43.647958,-79.396006,M5V 1K4,ON


In [39]:
# delete rows which its category is not Shopping Mall
df_Shopping = clean_Shopping_dataframe2[clean_Shopping_dataframe2.categories == 'Shopping Mall']
df_Shopping


,name,categories,address,lat,lng,postalCode,state
0,Dragon City Shopping Mall 龍城,Shopping Mall,280 Spadina Ave,43.652774,-79.398222,M5T 3A5,ON
1,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,43.653534,-79.380551,M5B 2H1,ON
2,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,43.647184,-79.380932,M5K 1A1,ON


#### generate map

In [42]:
# create dataframe of hotels, shopping stores and Cafeteria
hotel_neighbourhood_df = pd.concat([df_hotel, df_Cafeteria, df_Shopping], ignore_index=True)
hotel_neighbourhood_df

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
1,Grand Ballroom,Event Space,123 Queen St. W,43.651217,-79.383771,M5H 2M9,ON
2,Stay Cafeteria 慢走,Asian Restaurant,388 Spadina Ave,43.655454,-79.399163,M5T 2G5,ON
3,De La Salle College Cafeteria,College Cafeteria,131 Farnham Avenue,43.683003,-79.397815,M4V 1H7,ON
4,The Hub,College Cafeteria,350 Victoria St.,43.658585,-79.380622,M5B 2K3,ON
5,Marketeria,Restaurant,30 Bond St.,43.653585,-79.378843,M5B 1W8,ON
6,6th Floor Cafeteria 12 Concorde Place,Café,12 Concorde Place,43.721861,-79.329339,m3c 3k7,ON
7,Dragon City Shopping Mall 龍城,Shopping Mall,280 Spadina Ave,43.652774,-79.398222,M5T 3A5,ON
8,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,43.653534,-79.380551,M5B 2H1,ON
9,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,43.647184,-79.380932,M5K 1A1,ON


In [44]:
# Generate map to visualize hotel neighbourhood including shopping stores and Cafeteria 
hotel_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(hotel_neighbourhood_df['lat'], hotel_neighbourhood_df['lng'], 
                                           hotel_neighbourhood_df['name'], hotel_neighbourhood_df['categories'],\
                                               hotel_neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(hotel_map)  
    
hotel_map

In [47]:
# create dataframe of Park, Restaurant and Cafeteria
park_neighbourhood_df = pd.concat([df_park, df_Restaurant, df_Cafeteria,], ignore_index=True)
park_neighbourhood_df

,name,categories,address,lat,lng,postalCode,state
0,Queen's Park,Park,University Ave.,43.663946,-79.392180,M5R 2E8,ON
1,Canoe Landing Park,Park,50 Fort York Blvd,43.638762,-79.397067,M5V 3Z1,ON
2,Bellevue Square Park,Park,btwn Bellevue & Augusta Ave.,43.653610,-79.402199,M5T 2N4,ON
3,High Park,Park,1873 Bloor St. W,43.646479,-79.463425,M6R 2Z3,ON
4,Trinity Bellwoods Park,Park,1053 Dundas St. W.,43.647072,-79.413756,M5H 2N2,ON
5,Barbara Hall Park,Park,519 Church St,43.666879,-79.381068,M4Y 2K9,ON
6,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,M5G 1R3,ON
7,Rol San Restaurant 龍笙棧,Dim Sum Restaurant,323 Spadina Ave.,43.654318,-79.398650,M5T 2E9,ON
8,Swatow Restaurant 汕頭小食家,Chinese Restaurant,309 Spadina Ave.,43.653866,-79.398334,M5T 2E6,ON
9,Goldstone Noodle Restaurant 金石,Noodle House,266 Spadina Ave,43.652278,-79.398039,M5T 2E4,ON


In [48]:
# Generate map to visualize park neighbourhood including Restaurant and Cafeteria 
park_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(park_neighbourhood_df['lat'], park_neighbourhood_df['lng'], 
                                           park_neighbourhood_df['name'], park_neighbourhood_df['categories'],\
                                               park_neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(park_map)  
    
park_map

## 4. Results and Discussion <a name="results"></a>

Based on the standards and cluster data provided by the investor groups, the main neighborhood recommendation concerns the districts of the island. With a total score of 6.86, the area is located in category 0 (58 places / communities on average) and is a prime location for breweries / flagship restaurants.
A second recommendation was made to the community of Islington-City Center West. The community ranks first among the top five in the overall score evaluation and has less competition (only 36 sites within a one-mile radius).
In addition, it is evident from the grouping that the city of Toronto radiates outward from this point in an entertaining way in a circular form. Although not included in the research and analysis, occupancy rates for commercial square feet tend to follow this trend.


## 5. Conclusion

In short, the scope of this analysis is limited. The hotel industry is constantly evolving and the information provided to us may be outdated due to the reliance on user information through Foursquare. Overall, the data monitored through the Foursquare API and the data from the next census in 2021 make it easy to replicate the models created over and over. , We maintain our recommendations.